# 2 - Modelo Linear.ipynb — Modelo Preditivo Baseline (Regressão Linear)

Este notebook implementa a *primeira versão* de um modelo preditivo para estimar a **demanda horária** do pronto-socorro.

Seguiremos estas etapas:
1. Seleção das features e preparação dos dados
2. Divisão treino/teste respeitando series temporais
3. Treinamento do modelo baseline (Regressão Linear)
4. Predição
5. Avaliação com MAE, RMSE, MAPE e R²
6. Gráficos de diagnóstico

---

# 1. Importar bibliotecas e configurações iniciais

### 1.0 Instalações

In [38]:
# ! pip install scikit-learn

### 1.1 Importações

In [39]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error, mean_squared_error

import locale

### 1.2 Configurações de bibliotecas

In [40]:
# Desabilitar a limitação de linhas em gráficos do Altair
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [41]:
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

# 2. Importar e tratar dados

### 2.1 Importando dados

In [42]:
CAMINHO_DADOS = '../data/raw/dataset_pronto_socorro.csv'

df = pd.read_csv(CAMINHO_DADOS)

In [43]:
df.head()

,datetime,day_of_week,month,is_weekend,temperature,rain_mm,demand
0,2023-01-01 00:00:00,6,1,1,24.483571,0.353269,29.0
1,2023-01-01 01:00:00,6,1,1,21.308678,5.847757,30.0
2,2023-01-01 02:00:00,6,1,1,25.238443,1.141991,30.0
3,2023-01-01 03:00:00,6,1,1,29.615149,0.524987,33.0
4,2023-01-01 04:00:00,6,1,1,20.829233,0.820610,33.0


### 2.2 Tratamento de Dados

In [44]:
# Converter coluna de data/hora para datetime e definir índice
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')

In [45]:
# Criando features temporais
df['hour'] = df.index.hour
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['month'] = df.index.month

# 3 Desenvolvimento de Modelo Preditivo

### 3.1 Seleção das variáveis do modelo

In [46]:
# Seleção das features para o baseline

# Features : Variáveis utilizadas para prever a demanda
features = ['hour', 'day_of_week', 'is_weekend', 'month', 'temperature', 'rain_mm']

# Target: Variável que queremos prever
target = 'demand'

In [47]:
# Definindo os dataframes de features e target
X = df[features]
y = df[target]

### 3.2 Divisão dos dados em treino/teste

- **Para séries temporais, é fundamental respeitar a ordem cronológica dos eventos**

In [48]:
# Neste caso, está sendo utilizada uma separação simples, onde 80% dos dados são usados para treino e 20% para teste.
split_ratio = 0.8
split_point = int(len(df) * split_ratio)

In [49]:
# Divindo os dados em treino e teste
X_train, X_test = X.iloc[:split_point], X.iloc[split_point:]
y_train, y_test = y.iloc[:split_point], y.iloc[split_point:]

In [50]:
print(f"Registros treino: {locale.format_string('%d',len(X_train),grouping=True)}")
print(f"Registros teste:  {locale.format_string('%d',len(X_test),grouping=True)}")

Registros treino: 14.016
Registros teste:  3.505


### 3.3 Treinar Regressão Linear (Baseline)

In [51]:
# Treinando o modelo de regressão linear
model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [52]:
# Resultado do modelo
print("Intercepto:", model.intercept_)
print("Coeficientes:")
for feat, coef in zip(features, model.coef_):
    print(f"{feat}: {coef:.4f}")

Intercepto: 26.886682094892613
Coeficientes:
hour: -0.2375
day_of_week: 0.0307
is_weekend: 4.9953
month: -0.6637
temperature: 0.0050
rain_mm: 0.0115


### 3.4 Predições

In [53]:
y_pred = model.predict(X_test)

# 4. Análise detalhada do resultado

### 4.1 Criar dataframe com resultados

In [54]:
df_eval = pd.DataFrame({
    "y_true": y_test,
    "y_pred": y_pred
})

In [55]:
# Erro residual
# Residual positivo = modelo subestimou
# Residual negativo = modelo superestimou

df_eval["residual"] = df_eval["y_true"] - df_eval["y_pred"]

In [56]:
df_eval.head()

,y_true,y_pred,residual
datetime,,,
2024-08-07 00:00:00,20.0,21.773658,-1.773658
2024-08-07 01:00:00,19.0,21.524931,-2.524931
2024-08-07 02:00:00,20.0,21.271025,-1.271025
2024-08-07 03:00:00,16.0,21.066547,-5.066547
2024-08-07 04:00:00,21.0,20.795490,0.204510


## 4.2 Métricas de Avaliação

#### 4.2.1 Erro absoluto médio
https://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error

O **Erro Absoluto Médio (MAE)** mede, em média, o quanto as previsões do modelo se afastam dos valores reais, considerando apenas a magnitude do erro. Ele é calculado pela média das diferenças absolutas entre o valor observado e o valor predito:

$$\frac{1}{n}\sum_{i=0}^{n-1}|y_{i}-\hat{y}_{i}|$$

- Por trabalhar com valores absolutos, o MAE não penaliza tanto erros grandes quanto outras métricas, como o MSE. 
- Ele é intuitivo e fácil de interpretar: **indica “quanto o modelo erra”, em média, na mesma unidade da variável predita**.

**Como avaliar o MAE:**
- Quanto menor o MAE, melhor o desempenho do modelo. 
- Um bom valor depende da escala da variável-alvo, por isso é importante compará-lo com benchmarks simples (como médias móveis) e com outros modelos testados. 
- Também é útil analisar se o MAE é coerente com o nível de precisão desejado para o problema.

In [57]:
mae = mean_absolute_error(df_eval['y_true'], df_eval['y_pred'])
print("MAE:", mae)

MAE: 3.1658019959167865


#### 4.2.2 Raiz do Erro Quadrático Médio

https://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error

A **Raiz do Erro Quadrático Médio (RMSE)** mede o desvio médio entre valores observados e preditos, mas com uma característica importante: **ela penaliza mais fortemente erros maiores**, pois calcula primeiro o erro quadrático médio (MSE) e depois extrai sua raiz quadrada: 
$$ \sqrt{\frac{1}{n}\sum_{i=0}^{n-1}\left(y_{i}-\hat{y}_{i}\right)^2}$$

**Por elevar os erros ao quadrado, a métrica é mais sensível a valores discrepantes (outliers) e a previsões muito distantes do real.**

**Como avaliar o RMSE:**

- Assim como no MAE, valores menores indicam melhor desempenho. **Porém, por penalizar mais erros altos, a RMSE costuma ser maior que o MAE**. 
- É especialmente útil quando erros grandes são mais prejudiciais no contexto do problema. Compare o RMSE com outros modelos e com benchmarks para verificar se o desempenho está adequado à escala e à necessidade do projeto.


In [58]:
# RMSE penaliza erros grandes → ótimo para detectar picos que o modelo perdeu.
rmse = root_mean_squared_error(df_eval['y_true'], df_eval['y_pred'])
print("RMSE:", rmse)

RMSE: 3.929591882930698


#### 4.2.3 Erro Percentual Absoluto Médio

O **Erro Percentual Absoluto Médio (MAPE)** expressa, em termos percentuais, o quanto as previsões se afastam dos valores reais. Ele calcula a média dos erros absolutos divididos pelo valor observado, **permitindo interpretar o desempenho do modelo como um percentual de erro**:

$$\frac{100}{𝑛}\sum_{i=0}^{n-1}\left|\frac{y_i-\hat{y}_i}{y_i}\right|$$

Por ser uma métrica percentual, o MAPE facilita a comparação entre modelos e entre diferentes escalas de dados. No entanto, pode ser instável quando valores reais são muito próximos de zero, já que isso aumenta excessivamente o valor do erro relativo.

**Como avaliar o MAPE:**

- Quanto menor o percentual, melhor o modelo. Como referência geral, erros abaixo de 10% costumam indicar boa precisão, embora isso dependa do domínio do problema. 
- É importante verificar se não há muitos valores pequenos em y, pois eles distorcem o MAPE — caso isso ocorra, outras métricas podem ser mais adequadas.

In [59]:
# MAPE ajuda a entender o erro percentual, mas pode distorcer quando valores são pequenos.
mape = np.mean(np.abs(df_eval["residual"] / df_eval["y_true"])) * 100
print(f"MAPE: {mape:.2f}%")

MAPE: 18.95%


#### 4.2.4 Coeficiente de Determinação (R²)

O **Coeficiente de Determinação (R²)** indica quão bem o modelo consegue explicar a variabilidade dos dados reais. **Ele compara o desempenho do modelo com um modelo base que sempre prevê a média dos valores observados**. Sua fórmula geral é:

$$ 1 - \frac{\sum_{i=0}^{n-1}\left(y_{i}-\hat{y}_{i}\right)^2}{\sum_{i=0}^{n-1}\left(y_{i}-\bar{y}_{i}\right)^2}$$

Um valor de R² = 1 indica um ajuste perfeito, enquanto R² = 0 significa que o modelo não é melhor do que simplesmente prever a média. Valores negativos podem ocorrer quando o modelo é pior que o modelo base.

**Como avaliar o R²:**

- Quanto mais próximo de 1, melhor o modelo explica a variação dos dados. **Porém, é importante analisar o R² em conjunto com outras métricas, pois um R² alto não garante que as previsões individuais sejam precisas.** 
- Em séries temporais, especialmente, é comum que o R² seja mais baixo devido à complexidade dos padrões.

In [60]:
r2 = r2_score(df_eval['y_true'], df_eval['y_pred'])
print("R²:", r2)

R²: 0.24696353066566212


## 4.3 Distribuição dos Resíduos

In [61]:
# Se os resíduos não forem centrados em 0 → viés no modelo.
# Caudas pesadas → eventos extremos não capturados.

In [62]:
# Histograma
hist =alt.Chart(df_eval.reset_index()).mark_bar().encode(
    x=alt.X("residual", bin=alt.Bin(maxbins=50), title='Distribuição dos Resíduos'),
    y=alt.Y('count()',title='Frequência')
)
# Linha vertical no zero
linha_zero = (
    alt.Chart(pd.DataFrame({"x": [0]}))
    .mark_rule(color="black")
    .encode(x="x:Q")
)
# KDE (curva suavizada) — opcional
kde = (
    alt.Chart(df_eval.reset_index())
    .transform_density(
        "residual",
        as_=["residual", "density"]
    )
    .mark_line(color="red")
    .encode(
        x="residual:Q",
        y="density:Q"
    )
)

(hist + linha_zero + kde).resolve_scale(
    y="independent"
).properties(
    width=600,
    height=300,
    title="Distribuição dos resíduos"
)


alt.LayerChart(...)

### 4.4 Error por Hora do dia

- Este gráfico revela se o modelo falha no começo, meio ou final do dia.
- Se os erros forem sistematicamente altos em certas horas → falta de features temporais.

In [63]:
df_hour = df_eval.copy()
df_hour["hour"] = X_test["hour"]

hour_mae = df_hour.groupby("hour",as_index=False)["residual"].apply(lambda x: np.mean(np.abs(x)))
hour_mae.head()

,hour,residual
0,0,3.547716
1,1,3.499799
2,2,3.236725
3,3,3.405983
4,4,3.262916


In [73]:
chart = (
    alt.Chart(hour_mae)
    .mark_bar()
    .encode(
        x=alt.X("hour:O", title="Hora do dia"),
        y=alt.Y("residual:Q", title="Erro absoluto médio (MAE)"),
    )
    .properties(
        width=600,
        height=300,
        title="MAE por hora do dia"
    )
)

chart

alt.Chart(...)

### 4.5 Erro por dia da Semana

- A demanda hospitalar costuma variar muito entre domingo e segunda.
- Se o baseline não captura → será necessário adicionar lags.

In [67]:
df_dow = df_eval.copy()
df_dow["day_of_week"] = X_test["day_of_week"]

dow_mae = df_dow.groupby("day_of_week",as_index=False)["residual"].apply(lambda x: np.mean(np.abs(x)))
dow_mae.head()

,day_of_week,residual
0,0,3.163170
1,1,3.164267
2,2,3.210317
3,3,3.201825
4,4,3.117501


In [69]:
chart = (
    alt.Chart(dow_mae)
    .mark_bar()
    .encode(
        x=alt.X("day_of_week:O", title="Dia da Semana"),
        y=alt.Y("residual:Q", title="Erro absoluto médio (MAE)"),
    )
    .properties(
        width=600,
        height=300,
        title="MAE por dia da semana"
    )
)

chart

alt.Chart(...)

### 4.6 Comparação Real vs. Predição agregado por dia

- Linear Regression pode acertar nível médio mas errar amplitude.
- Se o modelo suaviza demais → pode exigir modelos não lineares.

In [ ]:
df_daily = df_eval[["y_true", "y_pred"]].resample("D").sum().reset_index()
df_daily.head()

,datetime,y_true,y_pred
0,2024-08-07,421.0,456.728481
1,2024-08-08,399.0,457.370599
2,2024-08-09,381.0,458.047106
3,2024-08-10,518.0,578.685940
4,2024-08-11,523.0,579.392136


In [ ]:
# Converte para formato long (necessário para múltiplas linhas no Altair)
df_long = df_daily.melt(id_vars="datetime", value_vars=["y_true", "y_pred"],
                       var_name="tipo", value_name="valor")
df_long.head()

,datetime,tipo,valor
0,2024-08-07,y_true,421.0
1,2024-08-08,y_true,399.0
2,2024-08-09,y_true,381.0
3,2024-08-10,y_true,518.0
4,2024-08-11,y_true,523.0


In [ ]:
chart = (
    alt.Chart(df_long)
    .mark_line()
    .encode(
        x=alt.X("datetime:T", title="Data"),
        y=alt.Y("valor:Q", title="Demanda"),
        color=alt.Color("tipo:N", title="Série", scale=alt.Scale(
            domain=["y_true", "y_pred"],
            range=["black", "steelblue"]
        )),
        tooltip=["data:T", "tipo:N", "valor:Q"]
    )
    .properties(
        width=1400,
        height=700,
        title="Demanda diária — Real vs Prevista"
    )
)

chart

alt.Chart(...)

### 4.7 Resíduos ao longo de tempo

- Ver se há períodos em que o modelo erra sistematicamente (viés temporal).
- Ver se há heterocedasticidade (erro aumenta em períodos de pico).

In [ ]:
# Linha dos resíduos
residual_line = (
    alt.Chart(df_eval.reset_index())
    .mark_line()
    .encode(
        x=alt.X("datetime:T", title="Data"),
        y=alt.Y("residual:Q", title="Resíduo")
    )
)

# Linha horizontal em zero
linha_zero = (
    alt.Chart(pd.DataFrame({"y": [0]}))
    .mark_rule(color="black")
    .encode(y="y:Q")
)

chart = (
    (residual_line + linha_zero)
    .properties(
        width=1200,
        height=400,
        title="Resíduos ao longo do tempo"
    )
)

chart

alt.LayerChart(...)

### 4.8 Interpretação dos coeficientes do modelo

- coef > 0 → aumenta demanda
- coef < 0 → reduz demanda
  
- Permite validar coerência com conhecimento hospitalar.

In [ ]:
coef_df = pd.DataFrame({
    "feature": features,
    "coef": model.coef_
})

coef_df.sort_values("coef", ascending=False)

,feature,coef
2,is_weekend,4.995281
1,dayofweek,0.030721
5,rain_mm,0.011516
4,temperature,0.005007
0,hour,-0.237546
3,month,-0.663652


### 4.9 Importância padronizada (coef*std)

- Isso mostra quais variáveis mais impactam a previsão na prática.
- Muito útil para justificar a evolução do modelo.

In [ ]:
stds = X_train.std()

coef_imp_df = pd.DataFrame({
    "feature": features,
    "coef": model.coef_,
    "std": stds,
})

coef_imp_df["importance_std"] = coef_imp_df["coef"] * coef_imp_df["std"]
coef_imp_df.sort_values("importance_std", ascending=False)

,feature,coef,std,importance_std
is_weekend,is_weekend,4.995281,0.451886,2.257298
dayofweek,dayofweek,0.030721,2.004345,0.061575
temperature,temperature,0.005007,5.009017,0.025078
rain_mm,rain_mm,0.011516,0.990856,0.011411
hour,hour,-0.237546,6.922434,-1.644400
month,month,-0.663652,3.227015,-2.141613


# 5. Conclusão

- Com os resultados obtidos, podemos notar que apesar de ser capaz de estimar o valor médio próximo, o modelo de regressão linear utilizando apenas estas informações básicas, falha em prever oscilações entre horas e dias.
- Apesar do bom resultado, mesmo para um modelo e modelagem de dados simples, temos como próximos passos para melhorar o resultado deste modelo, incorporar variáveis que auxiliem o modelo na identificação de padrões na série temporal, como janelas de médias e desvios móveis, bem como o histórico da demanda em algumas janelas de tempo.